In [8]:
#based on https://www.youtube.com/watch?v=eTieetk2dSw, https://www.philschmid.de/fine-tune-flan-t5-peft, https://medium.com/@ud.chandra/instruction-fine-tuning-llama-2-with-pefts-qlora-method-d6a801ebb19, 

from transformers import AutoModelForCausalLM, PreTrainedModel, AutoTokenizer, TrainingArguments
from dotenv import load_dotenv
import os
from datasets import load_from_disk
from peft import LoraConfig, PeftConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
import torch
from trl import SFTTrainer

In [9]:
#set wandb environment variables
os.environ["WANDB_ENTITY"] = "t_buess"
os.environ["WANDB_PROJECT"] = "chatbot-qa"

In [10]:
#load hugging-face token
load_dotenv()
hf_token = os.environ["HF_ACCESS_TOKEN"]

In [11]:
# Access the arguments
model_name = "meta-llama/Llama-2-7b-hf"
output_dir = "data/models/llama2-test"
ft_dataset_filename = "data/processed/ft_dataset.hf"
max_seq_length = 4096

In [12]:
def formatter(example):
    prompt = (
        "Nachfolgend ist eine Frage gestellt mit dem entsprechenden Kontext sowie der passenden Antwort"
        "Schreibe eine passende Antwort zur Frage und beziehe den Kontext mit hinein"
        "### Frage:\n"
        f"{example['question']}\n\n"
        "### Kontext:\n"
        f"{example['context']}\n\n"
        "### Antwort:\n"
        f"{example['answers']}"
    )

    return {"text": prompt}

#load train dataset
train_dataset = load_from_disk(ft_dataset_filename)

#add text column
train_dataset = train_dataset.map(formatter)

# for evaluation

In [13]:
query = "Was wird beim Unüberwachten Lernen gemacht?"
context = (
    "Unüberwachtes Lernen"
    "→ Hauptartikel: Unüberwachtes Lernen"
    "Der Algorithmus erzeugt für eine gegebene Menge von Eingaben ein statistisches Modell, das die Eingaben beschreibt und erkannte Kategorien und Zusammenhänge enthält und somit Vorhersagen ermöglicht. Dabei gibt es Clustering-Verfahren, die die Daten in mehrere Kategorien einteilen, die sich durch charakteristische Muster voneinander unterscheiden. Das Netz erstellt somit selbständig Klassifikatoren, nach denen es die Eingabemuster einteilt. Ein wichtiger Algorithmus in diesem Zusammenhang ist der EM-Algorithmus, der iterativ die Parameter eines Modells so festlegt, dass es die gesehenen Daten optimal erklärt. Er legt dabei das Vorhandensein nicht beobachtbarer Kategorien zugrunde und schätzt abwechselnd die Zugehörigkeit der Daten zu einer der Kategorien und die Parameter, die die Kategorien ausmachen. Eine Anwendung des EM-Algorithmus findet sich beispielsweise in den Hidden Markov Models (HMMs). Andere Methoden des unüberwachten Lernens, z. B. Hauptkomponentenanalyse, verzichten auf die Kategorisierung. Sie zielen darauf ab, die beobachteten Daten in eine einfachere Repräsentation zu übersetzen, die sie trotz drastisch reduzierter Information möglichst genau wiedergibt."
    "Des Weiteren unterscheidet man zwischen Batch-Lernen, bei dem alle Eingabe/Ausgabe-Paare gleichzeitig vorhanden sind, und kontinuierlichem (sequentiellem) Lernen, bei dem sich die Struktur des Netzes zeitlich versetzt entwickelt."
    "Außerdem unterscheidet man zwischen Off-line-Lernen, bei dem alle Daten gespeichert sind und somit wiederholbar zugreifbar sind, und On-line-Lernen, bei dem die Daten nach einmaligem Ausführen und Anpassen der Gewichte verloren gehen. Batch Training ist immer off-line, On-line-Training ist immer inkrementell. Inkrementelles Lernen kann jedoch on-line oder off-line erfolgen.[8]"
)

prompt = (
        "Nachfolgend ist eine Frage gestellt mit dem entsprechenden Kontext\n"
        "Schreibe eine passende Antwort zur Frage und beziehe den Kontext mit hinein\n\n"
        "### Frage:\n"
        f"{query}\n\n"
        "### Kontext:\n"
        f"{context}\n\n"
        "### Antwort:\n"
    )

# before fine tuning

In [14]:
model:PreTrainedModel = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="cuda:0",
    token=hf_token
)

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    token=hf_token
)
tokenizer.pad_token = "[PAD]"
tokenizer.padding_side = "right"

model.eval()

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda:0"), attention_mask=inputs["attention_mask"], max_new_tokens=200, pad_token_id=tokenizer.pad_token_id, do_sample=True)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

del model
del tokenizer

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.00s/it]
c:\Users\Stephan\anaconda3\envs\chatbot-qa\Lib\site-packages\transformers\utils\hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Nachfolgend ist eine Frage gestellt mit dem entsprechenden Kontext
Schreibe eine passende Antwort zur Frage und beziehe den Kontext mit hinein

### Frage:
Was wird beim Unüberwachten Lernen gemacht?

### Kontext:
Unüberwachtes Lernen→ Hauptartikel: Unüberwachtes LernenDer Algorithmus erzeugt für eine gegebene Menge von Eingaben ein statistisches Modell, das die Eingaben beschreibt und erkannte Kategorien und Zusammenhänge enthält und somit Vorhersagen ermöglicht. Dabei gibt es Clustering-Verfahren, die die Daten in mehrere Kategorien einteilen, die sich durch charakteristische Muster voneinander unterscheiden. Das Netz erstellt somit selbständig Klassifikatoren, nach denen es die Eingabemuster einteilt. Ein wichtiger Algorithmus in diesem Zusammenhang ist der EM-Algorithmus, der iterativ die Parameter eines Modells so festlegt, dass es die gesehenen Daten optimal erklärt. Er legt dabei das Vorhandensein nicht beobachtbarer Kategorien zugrunde und schätzt abwechselnd die Zugehörigkeit d

# inspiriert von (https://www.philschmid.de/fine-tune-flan-t5-peft)

In [15]:
# Define LoRA Config
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

In [16]:
model:PreTrainedModel = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="cuda:0",
    token=hf_token
)
model.config.use_cache = False
model = prepare_model_for_kbit_training(model)

# add LoRA adaptor
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.83s/it]


trainable params: 8,388,608 || all params: 6,746,804,224 || trainable%: 0.12433454005023165


In [17]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    token=hf_token
)
tokenizer.pad_token = "[PAD]"
tokenizer.padding_side = "right"

In [18]:
training_args = TrainingArguments(
    output_dir="data/processed/training-output",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    logging_steps=1,
    max_steps=400,
    report_to="wandb",
    fp16=True,
    optim="paged_adamw_8bit"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=lora_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_args,
)

trainer.train()

 96%|█████████▌| 382/400 [3:08:05<09:30, 31.69s/it]

{'loss': 1.134, 'learning_rate': 9e-06, 'epoch': 0.29}


 96%|█████████▌| 383/400 [3:08:30<08:21, 29.50s/it]

{'loss': 1.3336, 'learning_rate': 8.500000000000002e-06, 'epoch': 0.29}


 96%|█████████▌| 384/400 [3:09:01<07:58, 29.92s/it]

...

100%|█████████▉| 399/400 [3:15:38<00:26, 26.96s/it]

{'loss': 1.5246, 'learning_rate': 5.000000000000001e-07, 'epoch': 0.3}


100%|██████████| 400/400 [3:16:04<00:00, 29.41s/it]

{'loss': 1.3395, 'learning_rate': 0.0, 'epoch': 0.3}
{'train_runtime': 11767.5706, 'train_samples_per_second': 0.272, 'train_steps_per_second': 0.034, 'train_loss': 1.3600646409392356, 'epoch': 0.3}


TrainOutput(global_step=400, training_loss=1.3600646409392356, metrics={'train_runtime': 11767.5706, 'train_samples_per_second': 0.272, 'train_steps_per_second': 0.034, 'train_loss': 1.3600646409392356, 'epoch': 0.3})

In [19]:
#save model and tokenizer
trainer.model.save_pretrained(f"{output_dir}/model")
trainer.tokenizer.save_pretrained(f"{output_dir}/tokenizer")

('data/models/llama2-test/tokenizer\\tokenizer_config.json',
 'data/models/llama2-test/tokenizer\\special_tokens_map.json',
 'data/models/llama2-test/tokenizer\\tokenizer.model',
 'data/models/llama2-test/tokenizer\\added_tokens.json',
 'data/models/llama2-test/tokenizer\\tokenizer.json')

# after finetuning

In [20]:
model.eval()

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda:0"), attention_mask=inputs["attention_mask"], max_new_tokens=200, pad_token_id=tokenizer.pad_token_id, do_sample=True)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Nachfolgend ist eine Frage gestellt mit dem entsprechenden Kontext
Schreibe eine passende Antwort zur Frage und beziehe den Kontext mit hinein

### Frage:
Was wird beim Unüberwachten Lernen gemacht?

### Kontext:
Unüberwachtes Lernen→ Hauptartikel: Unüberwachtes LernenDer Algorithmus erzeugt für eine gegebene Menge von Eingaben ein statistisches Modell, das die Eingaben beschreibt und erkannte Kategorien und Zusammenhänge enthält und somit Vorhersagen ermöglicht. Dabei gibt es Clustering-Verfahren, die die Daten in mehrere Kategorien einteilen, die sich durch charakteristische Muster voneinander unterscheiden. Das Netz erstellt somit selbständig Klassifikatoren, nach denen es die Eingabemuster einteilt. Ein wichtiger Algorithmus in diesem Zusammenhang ist der EM-Algorithmus, der iterativ die Parameter eines Modells so festlegt, dass es die gesehenen Daten optimal erklärt. Er legt dabei das Vorhandensein nicht beobachtbarer Kategorien zugrunde und schätzt abwechselnd die Zugehörigkeit d

Struktur wurde erfolgreich übernommen.
Allerdings wiederholt das modell den Kontext.

# load model from storage

In [23]:
lora_config = LoraConfig.from_pretrained(f"{output_dir}/model")

tokenizer = AutoTokenizer.from_pretrained(f"{output_dir}/tokenizer")

model = AutoModelForCausalLM.from_pretrained(
    lora_config.base_model_name_or_path, 
    load_in_8bit=True,
    device_map="cuda:0",
    token=hf_token
)

model = get_peft_model(model, lora_config)

model.eval()

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda:0"), attention_mask=inputs["attention_mask"], max_new_tokens=200, pad_token_id=tokenizer.pad_token_id, do_sample=True)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.99s/it]


Nachfolgend ist eine Frage gestellt mit dem entsprechenden Kontext
Schreibe eine passende Antwort zur Frage und beziehe den Kontext mit hinein

### Frage:
Was wird beim Unüberwachten Lernen gemacht?

### Kontext:
Unüberwachtes Lernen→ Hauptartikel: Unüberwachtes LernenDer Algorithmus erzeugt für eine gegebene Menge von Eingaben ein statistisches Modell, das die Eingaben beschreibt und erkannte Kategorien und Zusammenhänge enthält und somit Vorhersagen ermöglicht. Dabei gibt es Clustering-Verfahren, die die Daten in mehrere Kategorien einteilen, die sich durch charakteristische Muster voneinander unterscheiden. Das Netz erstellt somit selbständig Klassifikatoren, nach denen es die Eingabemuster einteilt. Ein wichtiger Algorithmus in diesem Zusammenhang ist der EM-Algorithmus, der iterativ die Parameter eines Modells so festlegt, dass es die gesehenen Daten optimal erklärt. Er legt dabei das Vorhandensein nicht beobachtbarer Kategorien zugrunde und schätzt abwechselnd die Zugehörigkeit d